In [1]:
# importing libraries
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import geopandas
import io

## Loading Data

In [2]:
# geo_simple_parser.py
# Minimal parser for GEO Series Matrix (.txt) into two DataFrames:
#   sample_df (rows = samples, columns = metadata)
#   expr_df   (rows = features/probes, columns = samples)
#
# Notes:
# - Assumes !Sample_characteristics_ch1 rows like "key: value"
# - Uses !series_matrix_table_begin / !series_matrix_table_end to find the table
# - Does NOT need GeoPandas (GEO here is Gene Expression Omnibus)


def load_geo_series_matrix(path):
    # read all lines
    with open(path, encoding="utf-8", errors="replace") as f:
        lines = f.readlines()

    # --------- 1) collect sample metadata ---------
    # keep any !Sample_* lines
    sample_lines = [ln for ln in lines if ln.startswith("!Sample_")]

    # helper to split a tab line into fields (and strip quotes)
    def cells(line):
        parts = [p.strip().strip('"') for p in line.rstrip("\n").split("\t")]
        return parts

    # find number of samples (length of the first !Sample_* row minus the tag)
    n = max((len(cells(ln)) - 1 for ln in sample_lines), default=0)

    # start a dict of columns -> values
    meta = {}

    # capture a sample id column if present
    id_candidates = ["!Sample_geo_accession", "!Sample_title", "!Sample_name"]

    for ln in sample_lines:
        cols = cells(ln)
        tag, vals = cols[0], cols[1:]
        # pad short rows so everything has length n
        vals += [""] * (n - len(vals))

        # characteristics rows look like: "key: value"
        if tag.lower().startswith("!sample_characteristics"):
            # infer the column name from the first value's key
            first = vals[0]
            key = first.split(":", 1)[0].strip().lower().replace(" ", "_") if ":" in first else "characteristic"
            # keep only the value part
            vals = [v.split(":", 1)[1].strip() if ":" in v else v for v in vals]
            colname = key
        else:
            colname = tag[1:].lower()  # drop leading "!"

        # avoid overwriting duplicate column names
        while colname in meta:
            colname += "_dup"
        meta[colname] = vals

    # choose sample_id from common candidates or make generic names
    sample_id = None
    for tag in id_candidates:
        key = tag[1:].lower()
        if key in meta:
            sample_id = meta[key]
            break
    if sample_id is None:
        sample_id = [f"sample_{i+1}" for i in range(n)]
    meta["sample_id"] = sample_id

    sample_df = pd.DataFrame(meta).set_index("sample_id", drop=False)

    # --------- 2) read the expression table ---------
    # locate begin/end markers
    begin = next((i for i, ln in enumerate(lines) if ln.strip().lower() == "!series_matrix_table_begin"), None)
    end   = next((i for i, ln in enumerate(lines) if ln.strip().lower() == "!series_matrix_table_end"), None)

    if begin is None or end is None or end <= begin:
        # no table found -> empty frame
        expr_df = pd.DataFrame()
        return sample_df, expr_df

    table_text = "".join(lines[begin+1:end])
    expr_df = pd.read_csv(io.StringIO(table_text), sep="\t", dtype=str)
    expr_df = expr_df.set_index(expr_df.columns[0])       # first col = feature id
    expr_df.columns = [c.strip().strip('"') for c in expr_df.columns]
    expr_df = expr_df.apply(pd.to_numeric, errors="coerce")  # make numeric where possible

    return sample_df, expr_df

# quick demo (uncomment to run as a script):
# if __name__ == "__main__":
#     s, e = load_geo_series_matrix("GSE_series_matrix.txt")
#     print(s.head())
#     print(e.shape)


In [3]:
s, e = load_geo_series_matrix("raw_data/GSE121239_series_matrix.txt")
s.head()

,sample_title,sample_geo_accession,sample_status,sample_submission_date,sample_last_update_date,sample_type,sample_channel_count,sample_source_name_ch1,sample_organism_ch1,disease_state,...,sample_contact_institute,sample_contact_address,sample_contact_city,sample_contact_state,sample_contact_zip/postal_code,sample_contact_country,sample_supplementary_file,sample_data_row_count,sample_relation,sample_id
sample_id,,,,,,,,,,,,,,,,,,,,,
GSM3428310,PBMC_Healthy01,GSM3428310,Public on Oct 15 2018,Oct 15 2018,Oct 15 2018,RNA,1,PBMC_Healthy01,Homo sapiens,Healthy,...,GENYO,Avenida de la Ilustracion,GRANADA,Spain,18016,Spain,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3428...,54715,Reanalysis of: GSM1100843,GSM3428310
GSM3428311,PBMC_Healthy02,GSM3428311,Public on Oct 15 2018,Oct 15 2018,Oct 15 2018,RNA,1,PBMC_Healthy02,Homo sapiens,Healthy,...,GENYO,Avenida de la Ilustracion,GRANADA,Spain,18016,Spain,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3428...,54715,Reanalysis of: GSM1100844,GSM3428311
GSM3428312,PBMC_Healthy03,GSM3428312,Public on Oct 15 2018,Oct 15 2018,Oct 15 2018,RNA,1,PBMC_Healthy03,Homo sapiens,Healthy,...,GENYO,Avenida de la Ilustracion,GRANADA,Spain,18016,Spain,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3428...,54715,Reanalysis of: GSM1100845,GSM3428312
GSM3428313,PBMC_Healthy04,GSM3428313,Public on Oct 15 2018,Oct 15 2018,Oct 15 2018,RNA,1,PBMC_Healthy04,Homo sapiens,Healthy,...,GENYO,Avenida de la Ilustracion,GRANADA,Spain,18016,Spain,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3428...,54715,Reanalysis of: GSM1100846,GSM3428313
GSM3428314,PBMC_Healthy05,GSM3428314,Public on Oct 15 2018,Oct 15 2018,Oct 15 2018,RNA,1,PBMC_Healthy05,Homo sapiens,Healthy,...,GENYO,Avenida de la Ilustracion,GRANADA,Spain,18016,Spain,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3428...,54715,Reanalysis of: GSM1100847,GSM3428314


In [4]:
e.head()

,GSM3428310,GSM3428311,GSM3428312,GSM3428313,GSM3428314,GSM3428315,GSM3428316,GSM3428317,GSM3428318,GSM3428319,...,GSM3428612,GSM3428613,GSM3428614,GSM3428615,GSM3428616,GSM3428617,GSM3428618,GSM3428619,GSM3428620,GSM3428621
ID_REF,,,,,,,,,,,,,,,,,,,,,
1007_PM_s_at,4.955257,4.870351,4.889382,4.956931,4.518659,4.039076,4.363994,4.474612,4.109501,3.911813,...,4.143518,4.163726,3.552997,4.688624,4.101748,3.987773,4.674258,4.855265,5.227129,5.702039
1053_PM_at,5.984784,6.119432,5.914195,5.887814,5.962590,6.213503,6.523682,6.630676,6.424815,6.160038,...,5.601486,5.307239,5.585876,6.538760,6.420332,5.949633,5.587069,6.032248,6.317659,6.449117
117_PM_at,9.477945,9.828793,9.719707,9.148228,9.378308,9.688581,8.985668,10.329326,8.890190,9.604422,...,9.077410,9.349135,9.554533,9.941707,10.278584,9.672171,8.650779,9.264324,9.430051,9.660996
121_PM_at,4.553229,4.487615,4.645168,4.681996,4.990632,4.762670,4.528352,4.693206,3.875943,4.773145,...,5.177162,4.961182,5.267552,4.404966,4.013917,4.234865,4.418242,4.269110,4.115500,4.135403
1255_PM_g_at,1.921190,2.169639,2.410481,1.885852,1.987256,2.029976,2.110129,2.136902,2.169476,2.542144,...,2.224418,2.158327,2.316911,1.942149,1.929420,1.862300,1.842566,2.258901,1.951895,1.896087


## Cleaning Data

In [ ]:
#removing unnecessary collumns 
s.drop(columns=['sample_status', 'sample_status',
       'sample_submission_date', 'sample_last_update_date', 'sample_type',
       'sample_channel_count','sample_organism_ch1', 'sample_label_protocol_ch1', 'sample_taxid_ch1', 'sample_hyb_protocol',
       'sample_scan_protocol', 'sample_description', 'sample_data_processing',
       'sample_platform_id', 'sample_contact_name', 'sample_contact_email',
       'sample_contact_department', 'sample_contact_institute',
       'sample_contact_address', 'sample_contact_city', 'sample_contact_state',
       'sample_contact_zip/postal_code', 'sample_contact_country',
       'sample_supplementary_file', 'sample_data_row_count', 'sample_relation', 'sample_treatment_protocol_ch1','sample_growth_protocol_ch1', 'sample_molecule_ch1', 
       'sample_extract_protocol_ch1','sample_label_ch1', 'tissue'], inplace=True)

In [6]:
s['sledai'] = s['sledai'].astype(int)

In [7]:
#classificando as lesões e o caso do paciente: sledai: leve: 0-6, média 7-11, grave >=12

s['class'] = s['sledai'].apply(lambda x: 'leve' if 0 <= x <= 6 
                               else ('média' if 7 <= x <= 11 
                                             else 'grave'))
s.loc[s['sledai']==0, 'class'] = 'healthy'

In [8]:
#apenas pacientes com lupus 
s_lupus = s.loc[s['disease_state'] != 'Healthy' ]
s_lupus['patient_id'] = s_lupus['patient_id'].astype(int)
s_lupus = s_lupus.sort_values('patient_id')

/tmp/ipykernel_2331923/4106566585.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_lupus['patient_id'] = s_lupus['patient_id'].astype(int)


In [9]:
s_lupus

,sample_title,sample_geo_accession,sample_source_name_ch1,disease_state,patient_id,sledai,visit_date,imputed_neutrophil_percentage,sample_id,class
sample_id,,,,,,,,,,
GSM3428330,PBMC_SLE24_v1,GSM3428330,PBMC_SLE24_v1,Systemic Lupus Erythematosus,24,4,2009-10-15,89.3,GSM3428330,leve
GSM3428331,PBMC_SLE24_v2,GSM3428331,PBMC_SLE24_v2,Systemic Lupus Erythematosus,24,0,2009-12-07,68,GSM3428331,healthy
GSM3428332,PBMC_SLE24_v3,GSM3428332,PBMC_SLE24_v3,Systemic Lupus Erythematosus,24,8,2010-01-07,94.7,GSM3428332,média
GSM3428333,PBMC_SLE24_v4,GSM3428333,PBMC_SLE24_v4,Systemic Lupus Erythematosus,24,0,2010-02-19,64.9,GSM3428333,healthy
GSM3428334,PBMC_SLE24_v5,GSM3428334,PBMC_SLE24_v5,Systemic Lupus Erythematosus,24,0,2010-03-08,64.9,GSM3428334,healthy
...,...,...,...,...,...,...,...,...,...,...
GSM3428617,PBMC_SLE2129_v5,GSM3428617,PBMC_SLE2129_v5,Systemic Lupus Erythematosus,2129,2,2011-01-13,NA,GSM3428617,leve
GSM3428618,PBMC_SLE2132_v1,GSM3428618,PBMC_SLE2132_v1,Systemic Lupus Erythematosus,2132,2,2009-11-30,40.7,GSM3428618,leve
GSM3428619,PBMC_SLE2132_v2,GSM3428619,PBMC_SLE2132_v2,Systemic Lupus Erythematosus,2132,4,2010-02-22,44.6,GSM3428619,leve


In [10]:
df_patient = pd.DataFrame()
df_patient['patient_id'] = np.unique(s_lupus['patient_id'])

In [11]:
df_patient['first_visit'] = s_lupus.groupby('patient_id')['visit_date'].min().values
df_patient['last_visit'] = s_lupus.groupby('patient_id')['visit_date'].max().values

df_patient['min_sledai'] = s_lupus.groupby('patient_id')['sledai'].min().values
df_patient['max_sledai'] = s_lupus.groupby('patient_id')['sledai'].max().values

In [12]:
def classify_lesions(df, col_sledai, col_to): 
    df[col_to] = df[col_sledai].apply(lambda x: 'leve' if 0 <= x <= 6 
                               else ('média' if 7 <= x <= 11 
                                             else 'grave'))

In [13]:
s_lupus['visit_date'] = pd.to_datetime(s_lupus['visit_date'])

# ---- primeira visita ----
first_visits = s_lupus.sort_values('visit_date').groupby('patient_id').first()

# ---- última visita ----
last_visits = s_lupus.sort_values('visit_date').groupby('patient_id').last()

# ---- merge no df_patient ----
df_patient = df_patient.merge(
    first_visits[['sledai']], 
    left_on='patient_id', 
    right_index=True, 
    how='left'
).rename(columns={'sledai': 'first_sledai'})

df_patient['first_class'] = df_patient['first_sledai'].apply(lambda x: 'leve' if 0 <= x <= 6 
                               else ('média' if 7 <= x <= 11 
                                             else 'grave'))

df_patient = df_patient.merge(
    last_visits[['sledai']], 
    left_on='patient_id', 
    right_index=True, 
    how='left'
).rename(columns={'sledai': 'last_sledai'})

df_patient['last_class'] = df_patient['last_sledai'].apply(lambda x: 'leve' if 0 <= x <= 6 
                               else ('média' if 7 <= x <= 11 
                                             else 'grave'))

In [22]:
df_patient['max_sledai_cat'] = df_patient['max_sledai'].apply(lambda x: 'leve' if 0 <= x <= 6 
                               else ('média' if 7 <= x <= 11 
                                             else 'grave'))
df_patient['max_sledai_cat'].value_counts()

max_sledai_cat
leve     39
média    23
grave     3
Name: count, dtype: int64

In [19]:
df_patient['delta_sledai'] = df_patient['last_sledai'] - df_patient['first_sledai']

In [20]:
df_patient

,patient_id,first_visit,last_visit,min_sledai,max_sledai,first_sledai,first_class,last_sledai,last_class,delta_sledai
0,24,2009-10-15,2010-03-08,0,8,4,leve,0,leve,-4
1,46,2009-10-29,2010-08-26,0,10,2,leve,2,leve,0
2,113,2009-11-12,2010-07-29,0,2,0,leve,0,leve,0
3,244,2010-02-25,2011-11-10,0,10,0,leve,2,leve,2
4,317,2009-10-15,2011-10-31,2,10,2,leve,2,leve,0
...,...,...,...,...,...,...,...,...,...,...
60,2119,2009-09-28,2011-04-18,2,10,4,leve,10,média,6
61,2122,2010-07-29,2012-06-11,0,8,8,média,0,leve,-8
62,2128,2009-11-05,2011-11-03,2,4,4,leve,4,leve,0
63,2129,2009-10-15,2011-01-13,2,12,10,média,2,leve,-8


In [21]:
df_patient.loc[df_patient['last_class'] =='média']

,patient_id,first_visit,last_visit,min_sledai,max_sledai,first_sledai,first_class,last_sledai,last_class,delta_sledai
12,704,2009-12-14,2012-01-09,0,8,0,leve,8,média,8
40,1679,2009-10-22,2010-02-16,2,10,2,leve,10,média,8
46,1807,2009-10-29,2010-05-06,0,10,2,leve,10,média,8
60,2119,2009-09-28,2011-04-18,2,10,4,leve,10,média,6


In [22]:
df_patient['delta_sledai'].max()

np.int64(8)

In [23]:
s_lupus = s_lupus.sort_values('patient_id')
s_lupus.groupby("patient_id", group_keys=True)[['sledai', 'visit_date', 'class']].apply(lambda x: x)

sledai visit_date    class
patient_id sample_id                             
24         GSM3428330       4 2009-10-15     leve
           GSM3428331       0 2009-12-07  healthy
           GSM3428332       8 2010-01-07    média
           GSM3428333       0 2010-02-19  healthy
           GSM3428334       0 2010-03-08  healthy
...                       ...        ...      ...
2129       GSM3428617       2 2011-01-13     leve
2132       GSM3428618       2 2009-11-30     leve
           GSM3428619       4 2010-02-22     leve
           GSM3428620       4 2010-05-24     leve
           GSM3428621       2 2010-12-15     leve

[292 rows x 3 columns]

In [ ]:
#df_group = s_group.groupby('patient_id')['visit_date'].min()
#df_group = s_group.groupby('patient_id')['visit_date'].max()

NameError: name 's_group' is not defined

In [25]:
s_lupus.groupby('patient_id')['sledai'].max()

patient_id
24       8
46      10
113      2
244     10
317     10
        ..
2119    10
2122     8
2128     4
2129    12
2132     4
Name: sledai, Length: 65, dtype: int64

In [26]:
s_lupus.groupby('patient_id')['visit_date'].max()

patient_id
24     2010-03-08
46     2010-08-26
113    2010-07-29
244    2011-11-10
317    2011-10-31
          ...    
2119   2011-04-18
2122   2012-06-11
2128   2011-11-03
2129   2011-01-13
2132   2010-12-15
Name: visit_date, Length: 65, dtype: datetime64[ns]

In [27]:
df_patient = pd.DataFrame()
df_patient['patient_id'] = s_lupus.groupby('patient_id').index

AttributeError: 'DataFrameGroupBy' object has no attribute 'index'

In [ ]:
s_lupus.groupby('patient_id')['patient_id']

In [ ]:
s_lupus.groupby(as_index='patient_id', by='patient_id')[]

In [28]:
e.T

ID_REF,1007_PM_s_at,1053_PM_at,117_PM_at,121_PM_at,1255_PM_g_at,1294_PM_at,1316_PM_at,1320_PM_at,1405_PM_i_at,1431_PM_at,...,AFFX-r2-TagO-3_at,AFFX-r2-TagO-5_at,AFFX-r2-TagQ-3_at,AFFX-r2-TagQ-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
GSM3428310,4.955257,5.984784,9.477945,4.553229,1.921190,8.164253,5.152881,4.021098,8.082718,2.986367,...,2.983900,2.315920,2.284320,2.223882,3.513413,2.365662,2.277690,1.897895,2.335483,2.208698
GSM3428311,4.870351,6.119432,9.828793,4.487615,2.169639,7.886918,5.249574,3.965397,8.153894,2.792263,...,3.578466,2.407332,2.013488,2.153288,3.077168,2.344025,2.480338,1.632143,2.280212,2.123957
GSM3428312,4.889382,5.914195,9.719707,4.645168,2.410481,7.890372,4.735506,3.753827,8.105748,3.119576,...,3.689883,2.681990,2.684015,2.532046,3.346727,2.537787,2.475397,1.534480,2.479271,2.022023
GSM3428313,4.956931,5.887814,9.148228,4.681996,1.885852,8.006514,5.103395,3.939090,8.052309,3.324492,...,3.147272,2.513483,2.240368,1.962720,3.474484,2.140112,2.749770,1.484037,2.404451,2.013830
GSM3428314,4.518659,5.962590,9.378308,4.990632,1.987256,8.223370,5.122370,4.105659,8.274949,3.193524,...,3.160574,2.535126,2.330904,2.601260,3.229012,2.381164,2.593526,1.949685,2.560628,2.012157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM3428617,3.987773,5.949633,9.672171,4.234865,1.862300,7.906560,4.607779,4.037759,8.064507,2.957194,...,3.387968,2.942565,2.290730,2.120361,3.003179,2.252614,2.411526,1.661921,2.470162,2.132129
GSM3428618,4.674258,5.587069,8.650779,4.418242,1.842566,7.470552,4.788085,4.262485,8.945077,2.710398,...,3.160390,2.656507,2.232379,2.426584,3.146923,3.157310,2.435952,1.836948,2.490672,2.029212
GSM3428619,4.855265,6.032248,9.264324,4.269110,2.258901,7.490315,4.875119,3.878733,8.466693,2.977493,...,3.606283,2.378875,2.104084,1.747111,3.064181,2.106090,2.237814,1.777169,2.299285,2.355240
GSM3428620,5.227129,6.317659,9.430051,4.115500,1.951895,8.448537,4.472029,3.869092,9.018099,2.901236,...,3.153176,2.208815,2.224526,1.769736,2.888426,2.005743,2.377712,1.550851,2.253745,2.040009


## EDA

In [30]:
s.head()

,sample_title,sample_geo_accession,sample_source_name_ch1,disease_state,patient_id,sledai,visit_date,imputed_neutrophil_percentage,sample_id,class
sample_id,,,,,,,,,,
GSM3428310,PBMC_Healthy01,GSM3428310,PBMC_Healthy01,Healthy,NA,0,NA,NA,GSM3428310,healthy
GSM3428311,PBMC_Healthy02,GSM3428311,PBMC_Healthy02,Healthy,NA,0,NA,NA,GSM3428311,healthy
GSM3428312,PBMC_Healthy03,GSM3428312,PBMC_Healthy03,Healthy,NA,0,NA,NA,GSM3428312,healthy
GSM3428313,PBMC_Healthy04,GSM3428313,PBMC_Healthy04,Healthy,NA,0,NA,NA,GSM3428313,healthy
GSM3428314,PBMC_Healthy05,GSM3428314,PBMC_Healthy05,Healthy,NA,0,NA,NA,GSM3428314,healthy


In [24]:
s.columns

Index(['sample_title', 'sample_geo_accession', 'sample_source_name_ch1',
       'disease_state', 'patient_id', 'sledai', 'visit_date',
       'imputed_neutrophil_percentage', 'sample_id', 'class'],
      dtype='object')

In [41]:
s[s['patient_id'] == '317']

,sample_title,sample_geo_accession,sample_source_name_ch1,disease_state,patient_id,sledai,visit_date,imputed_neutrophil_percentage,sample_id,class
sample_id,,,,,,,,,,
GSM3428350,PBMC_SLE317_v1,GSM3428350,PBMC_SLE317_v1,Systemic Lupus Erythematosus,317,2,2009-10-15,86,GSM3428350,leve
GSM3428351,PBMC_SLE317_v2,GSM3428351,PBMC_SLE317_v2,Systemic Lupus Erythematosus,317,2,2009-11-12,92,GSM3428351,leve
GSM3428352,PBMC_SLE317_v3,GSM3428352,PBMC_SLE317_v3,Systemic Lupus Erythematosus,317,2,2009-12-17,NA,GSM3428352,leve
GSM3428353,PBMC_SLE317_v4,GSM3428353,PBMC_SLE317_v4,Systemic Lupus Erythematosus,317,2,2010-03-18,65,GSM3428353,leve
GSM3428354,PBMC_SLE317_v5,GSM3428354,PBMC_SLE317_v5,Systemic Lupus Erythematosus,317,2,2011-02-07,NA,GSM3428354,leve
GSM3428355,PBMC_SLE317_v6,GSM3428355,PBMC_SLE317_v6,Systemic Lupus Erythematosus,317,2,2011-05-11,NA,GSM3428355,leve
GSM3428356,PBMC_SLE317_v7,GSM3428356,PBMC_SLE317_v7,Systemic Lupus Erythematosus,317,10,2011-06-27,NA,GSM3428356,média
GSM3428357,PBMC_SLE317_v8,GSM3428357,PBMC_SLE317_v8,Systemic Lupus Erythematosus,317,2,2011-10-31,NA,GSM3428357,leve
